# Get the dataset from- 
http://people.dbmi.columbia.edu/~friedma/Projects/DiseaseSymptomKB/index.html

In [ ]:
# Import Dependencies
import csv
import pandas as pd
import numpy as np
from collections import defaultdict
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
disease_list = []

def return_list(disease):
    disease_list = []
    match = disease.replace('^','_').split('_') # using _ as common splitting delimeter
    ctr = 1
    for group in match:
        if ctr%2==0:
            disease_list.append(group) # refer the data format
        ctr = ctr + 1

    return disease_list

In [ ]:
with open("/content/Datasets/raw_data_2.csv") as csvfile:
    reader = csv.reader(csvfile)
    disease=""
    weight = 0
    disease_list = []
    dict_wt = {}
    dict_=defaultdict(list)

    for row in reader:

        if row[0]!="\xc2\0xa0" and row[0]!="": # for handling file encoding errors
          # saving disease and frequency
            disease = row[0]
            disease_list = return_list(disease)
            weight = row[1]

        if row[2]!="\xc2\0xa0" and row[2]!="":
            symptom_list = return_list(row[2])

            for d in disease_list:
                for s in symptom_list:
                    dict_[d].append(s) # adding all symptoms
                dict_wt[d] = weight

In [ ]:
with open("dataset_clean.csv","w") as csvfile:
    writer = csv.writer(csvfile)
    for key,values in dict_.items():
        for v in values:
            #key = str.encode(key)
            key = str.encode(key).decode('utf-8')
            #.strip()
            #v = v.encode('utf-8').strip()
            #v = str.encode(v)
            writer.writerow([key,v,dict_wt[key]])

columns = ['Source','Target','Weight'] # source: disease, target: symptom, weight: number of cases
data = pd.read_csv("dataset_clean.csv",names=columns, encoding ="ISO-8859-1")
data.head()


In [ ]:
data.to_csv("dataset_clean.csv",index=False)
data = pd.read_csv("dataset_clean.csv", encoding ="ISO-8859-1")
data.head()
len(data['Source'].unique()) # unique diseases
len(data['Target'].unique()) # unique symptoms
df = pd.DataFrame(data)
df_1 = pd.get_dummies(df.Target) # 1 hot encoding symptoms
df_1.head()
df.head()

df_s = df['Source']
df_pivoted = pd.concat([df_s,df_1], axis=1)
df_pivoted.drop_duplicates(keep='first',inplace=True)
df_pivoted[:5]


In [ ]:
len(df_pivoted)
cols = df_pivoted.columns
cols = cols[1:] # removing headings

# visualizing existance of symptoms for diseases
df_pivoted = df_pivoted.groupby('Source').sum()
df_pivoted = df_pivoted.reset_index()
df_pivoted[:5]


In [ ]:
len(df_pivoted)
df_pivoted.to_csv("df_pivoted.csv")
# defining data for training
x = df_pivoted[cols]
y = df_pivoted['Source']
# importing all needed libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split


In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)




In [ ]:
# Training multinomial naive bayes
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)

mnb.score(x_test, y_test)


mnb_tot = MultinomialNB()
mnb_tot = mnb_tot.fit(x, y)
mnb_tot.score(x, y)
disease_pred = mnb_tot.predict(x)
disease_real = y.values

In [ ]:
# printing model error
for i in range(0, len(disease_real)):
    if disease_pred[i]!=disease_real[i]:
        print ('Pred: {0} Actual:{1}'.format(disease_pred[i], disease_real[i]))


In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz

print ("DecisionTree")
dt = DecisionTreeClassifier()
clf_dt=dt.fit(x,y)
print ("Acurracy: ", clf_dt.score(x,y))

from sklearn import tree
from sklearn.tree import export_graphviz

export_graphviz(dt,
                out_file='tree.jpg',
                feature_names=cols
               )

from IPython.display import Image
Image(filename='tree.jpg')


In [ ]:
data = pd.read_csv("/content/Datasets/Training.csv")
data.head()
data.columns
len(data.columns)
len(data['prognosis'].unique())
df = pd.DataFrame(data)
df.head()
len(df)



In [ ]:
cols = df.columns
cols = cols[:-1]


In [ ]:

len(cols)
x = df[cols]
y = df['prognosis']

In [ ]:
x

y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
mnb = MultinomialNB()
mnb = mnb.fit(x_train, y_train)

mnb.score(x_test, y_test)

In [ ]:
from sklearn import model_selection
print ("cross result========")
scores = model_selection.cross_val_score(mnb, x_test, y_test, cv=3)
print (scores)
print (scores.mean())
test_data = pd.read_csv("/content/Datasets/Testing.csv")



In [ ]:
test_data.head()

In [ ]:
testx = test_data[cols]
testy = test_data['prognosis']

mnb.score(testx, testy)

from sklearn import model_selection
print ("cross result========")
scores = model_selection.cross_val_score(mnb, x_test, y_test, cv=3)
print (scores)
print (scores.mean())

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.33, random_state=42)
print ("DecisionTree")
dt = DecisionTreeClassifier(min_samples_split=20)
clf_dt=dt.fit(x_train,y_train)
print ("Acurracy: ", clf_dt.score(x_test,y_test))

from sklearn import model_selection
print ("cross result========")
scores = model_selection.cross_val_score(dt, x_test, y_test, cv=3)
print (scores)
print (scores.mean())


In [ ]:
print ("Acurracy on the actual test data: ", clf_dt.score(testx,testy))
from sklearn import tree
from sklearn.tree import export_graphviz

export_graphviz(dt,
                out_file='tree.dot',
                feature_names=cols)

!dot -Tpng tree.dot -o tree.png

from IPython.display import Image
Image(filename='tree.png')

dt.__getstate__()
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [ ]:

importances = dt.feature_importances_
print(dt.feature_importances_)


indices = np.argsort(importances)[::-1]
print([data.columns[i] for i in indices])
features = cols
for f in range(20):
    print("%d. feature %d - %s (%f)" % (f + 1, indices[f], features[indices[f]] ,importances[indices[f]]))



In [ ]:



export_graphviz(dt,
                out_file='tree-top5.dot',
                feature_names=cols,
                max_depth = 5
               )
!dot -Tpng tree-top5.dot -o tree-top5.png
from IPython.display import Image
Image(filename='tree-top5.png')


feature_dict = {}
for i,f in enumerate(features):
    feature_dict[f] = i

feature_dict['internal_itching']
sample_x = [i/52 if i==52 else i*0 for i in range(len(features))]
cols = list(data.columns)
print(cols.index('skin_rash'))

sample_x = np.array(sample_x).reshape(1,len(sample_x))
dt.predict(sample_x)

dt.predict_proba(sample_x)
len(sample_x)

In [ ]:
# symptoms = ['skin_rash','itching','nodal_skin_eruptions','increased_appetite','yellow_crust_ooze']
# ipt = [0 for i in range(len(features))]
# for s in symptoms:
#   ipt[cols.index(s)]=1
# ipt = np.array([ipt])
# print(ipt)
# print(dt.predict(ipt))
# dt.predict_proba(ipt)

# Display the list of all symptoms for the user to choose from
# Display the list of valid symptoms for user reference
print("Available Symptoms:")
valid_symptoms = [data.columns[i] for i in indices]
print(valid_symptoms)

# Initialize a list to collect the user-selected symptoms
input_symptoms = []


In [ ]:



# Loop to take user input for exactly 5 symptoms
print("\nEnter 5 symptoms one by one from the above list:")
for i in range(1, 6):  # Loop to take 5 symptoms
    symptom = input(f"Symptom {i}: ").strip()  # Take input for each symptom
    if symptom in valid_symptoms:  # Check if the symptom is valid
        input_symptoms.append(symptom)
    else:
        print(f"Invalid symptom: '{symptom}'. Please choose from the available symptoms.")
        exit()  # Exit the program if an invalid symptom is entered

# Convert symptoms into a binary input vector
ipt = [0 for _ in range(len(features))]
for symptom in input_symptoms:
    ipt[cols.index(symptom)] = 1
ipt = np.array([ipt])  # Reshape to make it compatible for prediction

In [ ]:

# Predict the disease based on the input symptoms
predicted_disease = dt.predict(ipt)
probabilities = dt.predict_proba(ipt)

# Display the prediction and probabilities
print("\nPrediction Results:")
print(f"Predicted Disease: {predicted_disease[0]}")
print("Prediction Probabilities:")
for i, prob in enumerate(probabilities[0]):
    print(f"{dt.classes_[i]}: {prob:.4f}")
